In [3]:
import os
import gc
#import cv2
import math
import copy
import time
import random
import glob

# Plotting
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
import seaborn as sns
from PIL import Image

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torchvision
from transformers import AutoImageProcessor, ResNetForImageClassification
from datasets import load_dataset
from torcheval.metrics.functional import binary_auroc
from torch.optim import lr_scheduler


from sklearn.model_selection import KFold, GroupKFold

import albumentations as A
from albumentations.pytorch import ToTensorV2

/home/mccruz/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
2024-08-09 21:03:44.313037: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-09 21:03:44.430250: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-09 21:03:46.083757: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO:albumentations.check_versi

In [4]:
## TODO

## Change ROOT_DIR, n_samples_train, n_samples_val, epochs, train_batch_size, valid_batch_size

In [5]:
ls | grep "v2"

v2_AUROC0.4981_Loss0.4338_epoch1.pth
v2_AUROC0.5030_Loss0.3580_epoch2.pth
v2_AUROC0.5357_Loss0.2494_epoch4.pth
v2_AUROC0.5720_Loss0.2480_epoch5.pth
v2_AUROC0.6107_Loss0.2470_epoch6.pth
v2_AUROC0.6293_Loss0.2448_epoch8.pth
v2_AUROC0.6331_Loss0.2455_epoch7.pth
v2_AUROC0.6440_Loss0.2437_epoch9.pth
v2_AUROC0.6632_Loss0.2425_epoch10.pth
v2_AUROC0.6685_Loss0.2392_epoch14.pth
v2_AUROC0.6687_Loss0.2404_epoch12.pth
v2_AUROC0.6718_Loss0.2415_epoch11.pth
v2_AUROC0.6737_Loss0.2385_epoch15.pth
v2_AUROC0.6776_Loss0.2398_epoch13.pth
v2_AUROC0.6788_Loss0.2353_epoch19.pth
v2_AUROC0.6798_Loss0.2371_epoch16.pth
v2_AUROC0.6809_Loss0.2322_epoch25.pth
v2_AUROC0.6827_Loss0.2312_epoch28.pth
v2_AUROC0.6836_Loss0.2304_epoch29.pth
v2_AUROC0.6838_Loss0.2328_epoch23.pth
v2_AUROC0.6848_Loss0.2347_epoch20.pth
v2_AUROC0.6856_Loss0.2303_epoch30.pth
v2_AUROC0.6856_Loss0.2362_epoch17.pth
v2_AUROC0.6871_Loss0.2325_epoch24.pth
v2_AUROC0.6901_Loss0.2311_epoch27.pth
v2_AUROC0.6917_Loss0.2328_epoch22.pth
v2_AUROC0.6919_Loss0

In [6]:
ROOT_DIR = "/home/mccruz/isic/ISIC2024_Skin_Cancer_Detection"
TRAIN_DIR = f'{ROOT_DIR}/train-image/image'

CONFIG = {
    "seed": 42,
    "n_samples_train":10000,
    "n_samples_val":10000, 
    "epochs": 50,
    "img_size": 384,
    "model_name": "tf_efficientnet_b0_ns",
    "checkpoint_path" : "/kaggle/input/tf-efficientnet/pytorch/tf-efficientnet-b0/1/tf_efficientnet_b0_aa-827b6e33.pth",
    "train_batch_size": 400,
    "valid_batch_size": 400,
    "learning_rate": 1e-4,
    "scheduler": 'CosineAnnealingLR',
    "min_lr": 1e-6,
    "T_max": 500,
    "weight_decay": 1e-6,
    "fold" : 4,
    "n_fold": 5,
    "n_accumulate": 1,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
}

BEST_WEIGHT = 'v2_AUROC0.9324_Loss0.0043_epoch22_lossauroc.pth'#'v2_AUROC0.8736_Loss0.0158_epoch12_lossauroc.pth'#'v2_AUROC0.6942_Loss0.2311_epoch26.pth'


In [7]:
CONFIG['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")#torch.device("cpu")

In [8]:
seed=CONFIG['seed']
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

## Data

In [9]:
train_df = pd.read_csv(ROOT_DIR+"/train-metadata.csv")
test_df = pd.read_csv(ROOT_DIR+"/test-metadata.csv")

all_df = pd.concat([train_df, test_df]).reset_index(drop=True)
display(train_df.head())
display(test_df.head())

/tmp/ipykernel_3346238/1934611744.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(ROOT_DIR+"/train-metadata.csv")


,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510


,isic_id,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,...,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license
0,ISIC_0015657,IP_6074337,45.0,male,posterior torso,2.70,TBP tile: close-up,3D: XP,22.80433,20.007270,...,0.304827,1.281532,2.299935,0.479339,20,-155.06510,1511.222000,113.980100,Memorial Sloan Kettering Cancer Center,CC-BY
1,ISIC_0015729,IP_1664139,35.0,female,lower extremity,2.52,TBP tile: close-up,3D: XP,16.64867,9.657964,...,0.000000,1.271940,2.011223,0.426230,25,-112.36924,629.535889,-15.019287,"Frazer Institute, The University of Queensland...",CC-BY
2,ISIC_0015740,IP_7142616,65.0,male,posterior torso,3.16,TBP tile: close-up,3D: XP,24.25384,19.937380,...,0.230742,1.080308,2.705857,0.366071,110,-84.29282,1303.978000,-28.576050,FNQH Cairns,CC-BY


In [10]:
train_images = sorted(glob.glob(f"{TRAIN_DIR}/*.jpg"))

In [11]:
## Images

def get_train_file_path(image_id):
    return f"{TRAIN_DIR}/{image_id}.jpg"

def show_im(image_id):
    image = mpimg.imread(image_id)
    plt.imshow(image)
    plt.show()

In [12]:
#for i in range(10):
#    image = mpimg.imread(train_images[i])
#    print(image.shape)
    

In [13]:
df = train_df.copy()
df['image_path'] = df['isic_id'].apply(get_train_file_path)
#df['image'] = df['isic_id'].apply(show_im)
df = df[ df["image_path"].isin(train_images) ].reset_index(drop=True)


print("# of images , # of positive cases, # of negative cases, # of patients")
print(df.shape, df.target.sum(), (df["target"] == 0).sum(), df["patient_id"].unique().shape)

df_positive = df[df["target"] == 1].reset_index(drop=True)
df_negative = df[df["target"] == 0].reset_index(drop=True)

# of images , # of positive cases, # of negative cases, # of patients
(401059, 56) 393 400666 (1042,)


## Start of Deep Learning: Pytorch

In [14]:
## CHANGE THIS, GOT THIS FROM COMMUNITY MODELS

data_transforms = {
    "train": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.RandomRotate90(p=0.5),
        A.Flip(p=0.5),
        A.Downscale(p=0.25),
        A.ShiftScaleRotate(shift_limit=0.1, 
                           scale_limit=0.15, 
                           rotate_limit=60, 
                           p=0.5),
        A.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
        A.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.),
    
    "validation": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.),
    
    "train_testing": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        #A.RandomRotate90(p=0.5),
        A.Flip(p=0.5),
        A.Downscale(p=0.25),
        A.ShiftScaleRotate(shift_limit=0.1, 
                           scale_limit=0.15, 
                           rotate_limit=60, 
                           p=0.5),
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
        A.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
        ToTensorV2()], p=1.)
}

In [15]:
class ISICDataset(Dataset):
    def __init__(self, df, phase="train", transforms=None):
        if phase == 'train':
            offset = CONFIG["n_samples_val"]
            n_samples = CONFIG["n_samples_train"]
        elif phase == 'train_testing':
            offset = CONFIG["n_samples_val"]
            n_samples = CONFIG["n_samples_train"]
        elif phase == 'validation':
            offset = 0
            n_samples = CONFIG["n_samples_val"]

        self.df_positive = df[df["target"] == 1].reset_index(drop=True)
        self.df_negative = df[df["target"] == 0].reset_index(drop=True)
        
        # Shuffle and slice the DataFrame
        self.df_negative = self.df_negative.sample(frac=1, random_state=42).reset_index(drop=True)
        self.df_negative = self.df_negative[offset:offset + n_samples // 2]

        self.transforms = transforms
        self.n_samples = n_samples

        # Generate multiple transformations for each positive sample
        self.positive_samples = []
        for idx in range(len(self.df_positive)):
            img_path = self.df_positive['image_path'].iloc[idx]
            img = Image.open(img_path).convert("RGB")
            img_np = np.array(img).copy()  # Ensure the array is copied
            for _ in range(n_samples // (2 * len(self.df_positive))):
                transformed_img = self.transforms(image=img_np.copy())["image"]
                self.positive_samples.append((transformed_img, 1))

        self.negative_samples = []
        for idx in range(len(self.df_negative)):
            img_path = self.df_negative['image_path'].iloc[idx]
            img = Image.open(img_path).convert("RGB")
            img_np = np.array(img).copy()  # Ensure the array is copied
            transformed_img = self.transforms(image=img_np)["image"]
            self.negative_samples.append((transformed_img, 0))

    def __len__(self):
        return self.n_samples

    def __getitem__(self, index):
        if random.random() < 0.5:
            idx = index % len(self.positive_samples)
            img, target = self.positive_samples[idx]
        else:
            idx = index % len(self.negative_samples)
            img, target = self.negative_samples[idx]

        return {
            'image': img,
            'target': target
        }


In [16]:
df

,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence,image_path
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282,/home/mccruz/isic/ISIC2024_Skin_Cancer_Detecti...
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455,/home/mccruz/isic/ISIC2024_Skin_Cancer_Detecti...
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,...,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040,/home/mccruz/isic/ISIC2024_Skin_Cancer_Detecti...
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,...,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998,/home/mccruz/isic/ISIC2024_Skin_Cancer_Detecti...
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,...,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510,/home/mccruz/isic/ISIC2024_Skin_Cancer_Detecti...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401054,ISIC_9999937,0,IP_1140263,70.0,male,anterior torso,6.80,TBP tile: close-up,3D: XP,22.574335,...,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999988,/home/mccruz/isic/ISIC2024_Skin_Cancer_Detecti...
401055,ISIC_9999951,0,IP_5678181,60.0,male,posterior torso,3.11,TBP tile: close-up,3D: white,19.977640,...,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999820,/home/mccruz/isic/ISIC2024_Skin_Cancer_Detecti...
401056,ISIC_9999960,0,IP_0076153,65.0,female,anterior torso,2.05,TBP tile: close-up,3D: XP,17.332567,...,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.999416,/home/mccruz/isic/ISIC2024_Skin_Cancer_Detecti...
401057,ISIC_9999964,0,IP_5231513,30.0,female,anterior torso,2.80,TBP tile: close-up,3D: XP,22.288570,...,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,/home/mccruz/isic/ISIC2024_Skin_Cancer_Detecti...


In [17]:
#train_dataset = ISICDataset(df, phase = "train", transforms=data_transforms["train"])
#valid_dataset = ISICDataset(df, phase = "validation", transforms=data_transforms["validation"])
train_dataset = ISICDataset(df, phase = "train", transforms=data_transforms["train_testing"])
valid_dataset = ISICDataset(df, phase = "validation", transforms=data_transforms["train_testing"])


train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                          num_workers=3, shuffle=True, pin_memory=True, drop_last=True)
valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                          num_workers=3, shuffle=False, pin_memory=True)

In [ ]:
'''
gkf = GroupKFold(n_splits=CONFIG['n_fold'])
for fold, (train_index, valid_index) in enumerate(gkf.split(train_df, train_df.target, train_df.patient_id)):
    train_df.loc[valid_index, "fold"] = int(fold)
    
display(train_df.groupby('fold').size()), 
display(train_df.head())
'''

In [19]:
## Model
#dataset = load_dataset("huggingface/cats-image")
#image = dataset["test"]["image"][0]

processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

#inputs = processor(image, return_tensors="pt")

#with torch.no_grad():
#    logits = model(**inputs).logits

## model predicts one of the 1000 ImageNet classes
#predicted_label = logits.argmax(-1).item()
#print(model.config.id2label[predicted_label])
new_classifier = nn.Sequential(
    nn.Flatten(start_dim=1, end_dim=-1),
    nn.Linear(in_features=2048, out_features=2, bias=True),
    nn.Sigmoid()
)

# Replace the old classifier with the new one
model.classifier = new_classifier

#for param in model.parameters():
#    param.requires_grad = False

    
for param in model.classifier.parameters():
    param.requires_grad = True

# Verify which parameters require gradients
#for name, param in model.named_parameters():
#    print(name, param.requires_grad)

model.load_state_dict( torch.load(BEST_WEIGHT,  map_location=torch.device('cpu')) )
model.to(CONFIG['device']);

In [20]:
#def criterion(outputs, targets):
#    return nn.MSELoss()(outputs, targets)
from sklearn.metrics import roc_curve, auc, roc_auc_score

def criterion(submission, solution, min_tpr: float=0.80) -> float:
    submission = submission[:,1]#.detach().numpy()
    solution = solution[:,1]#.detach().numpy()
    
    print(submission.shape, solution.shape)
    '''
    v_gt = abs(solution-1)
    v_pred = np.array([1.0 - x for x in submission])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return(partial_auc)
    '''
    
    gamma = 2
    alpha = 0.8

    BCE = F.binary_cross_entropy(submission, solution, reduction='mean')
    BCE_EXP = torch.exp(-BCE)
    focal_loss = alpha * (1-BCE_EXP)**gamma * BCE

    return focal_loss
    
    

In [21]:
#device = torch.device("cpu")

In [22]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    device = CONFIG["device"]
    model.train()

    dataset_size = 0
    running_loss = 0.0
    running_auroc  = 0.0

    for step, batch in enumerate(train_loader):
        print(step)
        images = batch['image'].to(device, dtype=torch.float)
        targets = batch['target'].to(device, dtype=torch.int64)  # Ensure targets are int64 for class indices

        
        batch_size = images.size(0)

        model = model.to(device)

        # Ensure images are properly processed
        inputs = processor(images, return_tensors="pt").to(device)

        # No need for torch.no_grad() during training
        logits = model(**inputs).logits  # Get raw logits

        # Choose the appropriate loss function
        loss = criterion(logits, torch.nn.functional.one_hot(targets, num_classes=2) * 1.0)  # Ensure targets are not one-hot encoded
        loss = loss / CONFIG['n_accumulate']

        print(loss)

        loss.backward()

        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()

        # Calculate AUROC after ensuring predicted labels are generated correctly
        predicted_label = torch.argmax(logits, dim=-1)  # Ensure to get class indices
        auroc = binary_auroc(input=predicted_label, target=targets).item()  # Ensure targets are class indices

        running_loss += (loss.item() * batch_size)
        running_auroc += (auroc * batch_size)
        dataset_size += batch_size

        train_epoch_loss = running_loss / dataset_size
        train_epoch_auroc = running_auroc / dataset_size

    gc.collect()

    return train_epoch_loss, train_epoch_auroc

In [23]:
@torch.inference_mode()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    running_auroc = 0.0
    
    for batch in valid_loader:   
        images = batch['image'].to(device, dtype=torch.float)
        targets = batch['target'].to(device, dtype=torch.int64)  # Ensure targets are int64 for class indices

        
        batch_size = images.size(0)

        model = model.to(device)

        # Ensure images are properly processed
        inputs = processor(images, return_tensors="pt").to(device)

        # No need for torch.no_grad() during training
        logits = model(**inputs).logits  # Get raw logits

        # Choose the appropriate loss function
        loss = criterion(logits, torch.nn.functional.one_hot(targets, num_classes=2) * 1.0)  # Ensure targets are not one-hot encoded
        loss = loss / CONFIG['n_accumulate']

        
        print(loss)

        predicted_label = torch.argmax(logits, dim=-1)  # Ensure to get class indices
        auroc = binary_auroc(input=predicted_label, target=targets).item()  # Ensure targets are class indices

        running_loss += (loss.item() * batch_size)
        running_auroc  += (auroc * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        epoch_auroc = running_auroc / dataset_size
        
      
    gc.collect()
    
    return epoch_loss, epoch_auroc

In [24]:
if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))

In [25]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [26]:
device = torch.device("cuda") 

In [27]:
    optimizer = optim.Adam(model.parameters(), lr=CONFIG['learning_rate'], 
                       weight_decay=CONFIG['weight_decay'])
    scheduler = fetch_scheduler(optimizer)

In [28]:
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_auroc = -np.inf
    history = {"Train Loss": [], "Valid Loss": [], 'Train AUROC': [], 'Valid AUROC' : [], 'lr' : []}
    

In [ ]:
    

    #start = time.time()
    #best_model_wts = copy.deepcopy(model.state_dict())
    #best_epoch_auroc = -np.inf
    #history = {"Train Loss": [], "Valid Loss": [], 'Train AUROC': [], 'Valid AUROC' : [], 'lr' : []}
    
    for epoch in range(13,13+CONFIG['epochs']): 
        gc.collect()
        train_epoch_loss, train_epoch_auroc = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss, val_epoch_auroc = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        history['Train AUROC'].append(train_epoch_auroc)
        history['Valid AUROC'].append(val_epoch_auroc)
        history['lr'].append( scheduler.get_lr()[0] )
        
        print(history)
        # deep copy the model
        if 2>1:# best_epoch_auroc <= val_epoch_auroc:
            print(f"Validation AUROC Improved ({best_epoch_auroc} ---> {val_epoch_auroc})")
            best_epoch_auroc = val_epoch_auroc
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = "/home/mccruz/isic/ISIC2024_Skin_Cancer_Detection/v2_AUROC{:.4f}_Loss{:.4f}_epoch{:.0f}_lossauroc.pth".format(val_epoch_auroc, val_epoch_loss, epoch)
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best AUROC: {:.4f}".format(best_epoch_auroc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    

0
torch.Size([400]) torch.Size([400])
tensor(0.0167, grad_fn=<DivBackward0>)
1
torch.Size([400]) torch.Size([400])
tensor(0.0169, grad_fn=<DivBackward0>)
2
torch.Size([400]) torch.Size([400])
tensor(0.0119, grad_fn=<DivBackward0>)
3
torch.Size([400]) torch.Size([400])
tensor(0.0177, grad_fn=<DivBackward0>)
